# CNN Using MNIST Data Set

In [2]:
import tensorflow as tf

### Import MNIST Data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Functions:
- Initialize the weights of a layer
- Initialize the bias of a layer
- Return a 2D convolution
- Pooling

In [5]:
# Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

In [6]:
# Bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape = shape)
    return tf.Variable(init_bias_vals)

In [7]:
# 2D convolution
def conv2d(x,W):
    # x -> [batch, Height, Width, Channel(grayscale)]
    # W -> [filter Height, filter Width, Channels IN, Channels OUT]
    
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = 'SAME')

In [8]:
# Pooling
def max_pool_2by2(x):
    # x -> [batch, Height, Width, Channel]
    return tf.nn.max_pool(x, ksize = [1,2,2,1],strides = [1,2,2,1], padding = 'SAME')

In [9]:
# Convolutional layer
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    # Activation function ReLu
    return tf.nn.relu(conv2d(input_x, W) + b)

In [10]:
# Normal fully connected layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Build CNN Model

In [11]:
# Placeholders
x = tf.placeholder(tf.float32, shape = [None, 784])

In [12]:
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [13]:
# Layers 
x_image = tf.reshape(x, [-1,28,28,1]) # Input image

In [14]:
# 32 features per 5x5 patch grayscale (1-channel)
convo_1 = convolutional_layer(x_image, shape = [5,5,1,32]) 
convo_1_pooling = max_pool_2by2(convo_1)

In [15]:
convo_2 = convolutional_layer(convo_1_pooling, shape = [5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [16]:
# Flatten layer
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [17]:
# Dropout 
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [18]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [19]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits =y_pred))

In [20]:
# Optimizer - Adam
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [21]:
init = tf.global_variables_initializer()

In [22]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        # 50% dropout probability
        sess.run(train, feed_dict = {x: batch_x, y_true: batch_y, hold_prob: 0.5})
        
        # Check accuracy every 100 steps
        if i%100 == 0:
            
            print("On Step: {}".format(i))
            print("Accuracy: ")
            
            # Calculate + cast
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc, feed_dict = {x:mnist.test.images,y_true: mnist.test.labels,hold_prob:1.0}))
            print('\n')
            

On Step: 0
Accuracy: 
0.1102


On Step: 100
Accuracy: 
0.944


On Step: 200
Accuracy: 
0.9637


On Step: 300
Accuracy: 
0.9679


On Step: 400
Accuracy: 
0.9713


On Step: 500
Accuracy: 
0.98


On Step: 600
Accuracy: 
0.9794


On Step: 700
Accuracy: 
0.9794


On Step: 800
Accuracy: 
0.9799


On Step: 900
Accuracy: 
0.98


On Step: 1000
Accuracy: 
0.9844


On Step: 1100
Accuracy: 
0.9826


On Step: 1200
Accuracy: 
0.9841


On Step: 1300
Accuracy: 
0.9885


On Step: 1400
Accuracy: 
0.981


On Step: 1500
Accuracy: 
0.986


On Step: 1600
Accuracy: 
0.989


On Step: 1700
Accuracy: 
0.9871


On Step: 1800
Accuracy: 
0.9872


On Step: 1900
Accuracy: 
0.9884


On Step: 2000
Accuracy: 
0.9893


On Step: 2100
Accuracy: 
0.9852


On Step: 2200
Accuracy: 
0.9879


On Step: 2300
Accuracy: 
0.9891


On Step: 2400
Accuracy: 
0.9879


On Step: 2500
Accuracy: 
0.9873


On Step: 2600
Accuracy: 
0.9883


On Step: 2700
Accuracy: 
0.9883


On Step: 2800
Accuracy: 
0.989


On Step: 2900
Accuracy: 
0.9882


O

### ~99.04% accuracy on model